In [1]:
import numpy as np
import pandas as pd
import os
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
import pickle
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
es_url = 'http://ckg07:9200'
es_index = 'wikidatadwd-augmented'
index_url = 'http://ckg07:9200/wikidatadwd-augmented/'

HOME_DIR = '/home/sriamazingram/USC/Others/ISI/data/t2dv2-example'

# Input Paths

# GDrive Path: https://drive.google.com/drive/folders/1ESBZCgILSRnH1-jgvqpY2dKcj19MLtlq?usp=sharing
tables_path = f"{HOME_DIR}/t2dv2-dev-input"

# GDrive Path: https://drive.google.com/drive/folders/1gQ15t8T1HW9j1zJfThZNyk7Wfxo0-uy5?usp=sharing
gt_path = f'{HOME_DIR}/gt'

# GDrive Path: https://drive.google.com/file/d/1fC5zyD9v5KnvGY1cY1FAAe2XvJLNdxCX/view?usp=sharing
targets_columns = f"{HOME_DIR}/Files1.txt"

# OUTPUT PATHS

canonical_path = f'{HOME_DIR}/canonical'
candidate_path = f'{HOME_DIR}/candidates'
feature_path = f'{HOME_DIR}/features'

temp_dir = f'{HOME_DIR}/temp'

output_predictions = f'{HOME_DIR}/dev_predictions'
predictions_top_k = f'{HOME_DIR}/dev_predictions_top_k'
colorized_path = f'{HOME_DIR}/dev_predictions_colorized'
metrics_path = f'{HOME_DIR}/dev_predictions_metrics'

aux_field = 'graph_embedding_complex,class_count,property_count,context'

prop_count = f'{HOME_DIR}/property_count' 
class_count = f'{HOME_DIR}/class_count'
context_path = f'{HOME_DIR}/context'
embedding_path = f'{HOME_DIR}/embedding'

# MODEL PATHS

#GitHub Link to models: https://github.com/usc-isi-i2/table-linker-pipelines/tree/main/table-linker-full-pipeline/models
model_file_path = './models/weighted_lr.pkl'
ranking_model_file_path = './models/epoch_5_loss_0.09882864356040955_top1_0.8968926553672316.pth'
min_max_scaler_path = './models/normalization_factor.pkl'

final_score_column = "siamese_prediction"

In [3]:
features = ['pagerank','retrieval_score','monge_elkan','monge_elkan_aliases','des_cont_jaccard',
            'jaro_winkler','levenshtein','singleton','num_char','num_tokens',
           'lof_class_count_tf_idf_score', 'lof_property_count_tf_idf_score',
           'lof-graph-embedding-score', 'lof-reciprocal-rank', 'context_score']

### Create folders for storing files

In [4]:
!mkdir -p $temp_dir
!mkdir -p $canonical_path
!mkdir -p $candidate_path
!mkdir -p $feature_path
!mkdir -p $output_predictions
!mkdir -p $predictions_top_k
!mkdir -p $colorized_path
!mkdir -p $metrics_path
!mkdir -p $prop_count
!mkdir -p $class_count
!mkdir -p $context_path
!mkdir -p $embedding_path

### Canonicalize

In [5]:
targets = pd.read_csv(targets_columns, header=None, index_col=0)
for file in glob.glob(tables_path+"/*.csv"):
    
    filename = file.split("/")[-1].split(".csv")[0]
    column_name = targets.loc[filename][1]
    canonical_file = f'{canonical_path}/{file.split("/")[-1]}'
    print(f"File:{filename} - Column:{column_name}")
    
    !tl canonicalize $file -c "$column_name" --add-context > $canonical_file

File:84575189_0_6365692015941409487 - Column:Journal
canonicalize Time: 0.0036840438842773438s
File:28086084_0_3127660530989916727 - Column:Saint
canonicalize Time: 0.004144906997680664s
File:50270082_0_444360818941411589 - Column:Player (2011 Ws)
canonicalize Time: 0.011132478713989258s
File:29414811_2_4773219892816395776 - Column:Game
canonicalize Time: 0.0030183792114257812s
File:39759273_0_1427898308030295194 - Column:Title
canonicalize Time: 0.0033702850341796875s
File:14380604_4_3329235705746762392 - Column:Company
canonicalize Time: 0.0032808780670166016s
File:1438042986423_95_20150728002306-00329-ip-10-236-191-2_805336391_10 - Column:Name
canonicalize Time: 0.003033876419067383s
File:14067031_0_559833072073397908 - Column:OAU State
canonicalize Time: 0.0035049915313720703s
File:45073662_0_3179937335063201739 - Column:Player (2011 TBs)
canonicalize Time: 0.0027573108673095703s
File:52299421_0_4473286348258170200 - Column:1998 Data Country
canonicalize Time: 0.00403594970703125s


### Candidate Generation

In [7]:
for file in glob.glob(canonical_path+"/*.csv"):
    
    filename = file.split("/")[-1]
    candidate_file_path = f'{candidate_path}/{filename}'
    gt_file = f'{gt_path}/{filename}'
    
    print(filename)
    
    !tl clean -c label -o label_clean "$file" \
    / --url "$es_url" --index "$es_index" \
    get-fuzzy-augmented-matches -c label_clean \
    --auxiliary-fields "$aux_field" \
    --auxiliary-folder "$temp_dir" \
    / --url "$es_url" --index "$es_index" \
    get-exact-matches \
    -c label_clean --auxiliary-fields "$aux_field" \
    --auxiliary-folder "$temp_dir" \
    / ground-truth-labeler --gt-file $gt_file > "$candidate_file_path"
    
    for field in aux_field.split(','):
        aux_list = []
        for f in glob.glob(f'{temp_dir}/*{field}.tsv'):
            aux_list.append(pd.read_csv(f, sep='\t', dtype=object))
        aux_df = pd.concat(aux_list).drop_duplicates(subset=['qnode'])
        if field == 'class_count':
            class_count_file = f"{class_count}/{filename.strip('.csv')}_class_count.tsv"
            aux_df.to_csv(class_count_file, sep='\t', index=False)
        elif field == 'property_count':
            prop_count_file = f"{prop_count}/{filename.strip('.csv')}_prop_count.tsv"
            aux_df.to_csv(prop_count_file, sep='\t', index=False)
        elif field == 'context':
            context_file = f"{context_path}/{filename.strip('.csv')}_context.tsv"
            aux_df.to_csv(context_file, sep='\t', index=False)
        elif field == 'graph_embedding_complex':
            embedding_file = f"{embedding_path}/{filename.strip('.csv')}_embedding.tsv"
            aux_df.to_csv(embedding_file, sep='\t', index=False)

84575189_0_6365692015941409487.csv
clean Time: 0.005683183670043945s
get-fuzzy-augmented-matches Time: 13.465822696685791s
get-exact-matches Time: 5.890868902206421s
ground-truth-labeler Time: 0.22819805145263672s
28086084_0_3127660530989916727.csv
clean Time: 0.009492158889770508s
get-fuzzy-augmented-matches Time: 20.001757383346558s
get-exact-matches Time: 16.32868981361389s
ground-truth-labeler Time: 0.40917015075683594s
50270082_0_444360818941411589.csv
clean Time: 0.0070629119873046875s
get-fuzzy-augmented-matches Time: 18.000375270843506s
get-exact-matches Time: 6.323396682739258s
ground-truth-labeler Time: 0.2840452194213867s
29414811_2_4773219892816395776.csv
clean Time: 0.0017108917236328125s
get-fuzzy-augmented-matches Time: 7.4294703006744385s
get-exact-matches Time: 0.40150976181030273s
ground-truth-labeler Time: 0.052536964416503906s
39759273_0_1427898308030295194.csv
clean Time: 0.0049190521240234375s
get-fuzzy-augmented-matches Time: 11.664368629455566s
get-exact-matches

### Feature Generation

In [8]:
for file in glob.glob(candidate_path+"/*.csv"):
    
    filename = file.split("/")[-1]
    feature_file = f'{feature_path}/{filename}'
    context_file = f'{context_path}/{filename.strip(".csv")}_context.tsv'
    graph_embedding_complex_file = f'{embedding_path}/{filename.strip(".csv")}_embedding.tsv'
    class_count_file = f'{class_count}/{filename.strip(".csv")}_class_count.tsv'
    property_count_file = f'{prop_count}/{filename.strip(".csv")}_prop_count.tsv'
    
    if filename == "1438042986423_95_20150728002306-00329-ip-10-236-191-2_805336391_10.csv":
        continue
    print(filename)
    # Aligned page rank
    # String similarity features
    # Normalize Jaccard similarity
    # Calculate mosaic features
    # Singleton feature
    # Most voted candidates result
    # Graph-embedding-score using centroid-of-lof and lof-strategy
    # LOF reciprocal rank feature
    # LOF TF-IDF feature for classes and properties
    # Context Match feature
    
    !tl align-page-rank $file \
    / string-similarity -i --method symmetric_monge_elkan:tokenizer=word -o monge_elkan \
    / string-similarity -i --method symmetric_monge_elkan:tokenizer=word -c label_clean kg_aliases -o monge_elkan_aliases \
    / string-similarity -i --method jaro_winkler -o jaro_winkler \
    / string-similarity -i --method levenshtein -o levenshtein \
    / string-similarity -i --method jaccard:tokenizer=word -c kg_descriptions context -o des_cont_jaccard \
    / normalize-scores -c des_cont_jaccard / smallest-qnode-number \
    / mosaic-features -c kg_labels --num-char --num-tokens \
    / create-singleton-feature -o singleton \
    / vote-by-classifier \
    --prob-threshold 0.995 \
    --model $model_file_path \
    --features "aligned_pagerank,smallest_qnode_number,monge_elkan,des_cont_jaccard_normalized" \
    / score-using-embedding \
    --column-vector-strategy centroid-of-lof \
    --lof-strategy ems-mv \
    -o lof-graph-embedding-score \
    --embedding-file $graph_embedding_complex_file \
    --embedding-url $index_url \
    / generate-reciprocal-rank \
    -c lof-graph-embedding-score \
    -o lof-reciprocal-rank \
    / compute-tf-idf \
    --feature-file "$class_count_file" \
    --feature-name class_count \
    --singleton-column is_lof \
    -o lof_class_count_tf_idf_score \
    / compute-tf-idf \
    --feature-file "$property_count_file" \
    --feature-name property_count \
    --singleton-column is_lof \
    -o lof_property_count_tf_idf_score \
    / context-match \
    --context-file $context_file  \
    -o context_score \
    --debug \
    > "$feature_file"

84575189_0_6365692015941409487.csv
align-page-rank Time: 0.678173303604126s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 5.831399440765381s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 11.020357370376587s
string-similarity-['jaro_winkler'] Time: 0.9630706310272217s
string-similarity-['levenshtein'] Time: 7.7811009883880615s
string-similarity-['jaccard:tokenizer=word'] Time: 0.10764527320861816s
normalize-scores-des_cont_jaccard Time: 0.04472160339355469s
smallest-qnode-number Time: 0.4096338748931885s
mosaic-features Time: 0.01918482780456543s
create-singleton-feature Time: 0.2456960678100586s
/home/sriamazingram/USC/Others/ISI/tl_env/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
vote-by-classifier Time: 0.3747282028198242s
Qnodes to lookup: 8831


context-match Time: 14.177937507629395s
14067031_0_559833072073397908.csv
align-page-rank Time: 0.32572340965270996s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 3.1700901985168457s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 3.6154298782348633s
string-similarity-['jaro_winkler'] Time: 0.7208642959594727s
string-similarity-['levenshtein'] Time: 3.8121352195739746s
string-similarity-['jaccard:tokenizer=word'] Time: 0.20200133323669434s
normalize-scores-des_cont_jaccard Time: 0.035929203033447266s
smallest-qnode-number Time: 0.2347867488861084s
mosaic-features Time: 0.01794910430908203s
create-singleton-feature Time: 0.18232131004333496s
/home/sriamazingram/USC/Others/ISI/tl_env/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
vote-by-classifier Time:

### Model Prediction

In [9]:
features_str = ",".join(features)
for file in glob.glob(feature_path+"/*.csv"):
    filename = file.split("/")[-1]
    
    print(filename)
    
    prediction_file = f'{output_predictions}/{filename}'
    
    features_str = ",".join(features)
    !tl predict-using-model -o siamese_prediction \
    --ranking-model $ranking_model_file_path \
    --features $features_str \
    --normalization-factor $min_max_scaler_path $file > $prediction_file

84575189_0_6365692015941409487.csv
/home/sriamazingram/USC/Others/ISI/tl_env/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
predict-using-model Time: 1.0456326007843018s
28086084_0_3127660530989916727.csv
/home/sriamazingram/USC/Others/ISI/tl_env/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
predict-using-model Time: 1.9411087036132812s
50270082_0_444360818941411589.csv
/home/sriamazingram/USC/Others/ISI/tl_env/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or inval

### Get Top 5 Links

In [10]:
for file in glob.glob(output_predictions+"/*.csv"):
    filename = file.split("/")[-1]
    
    print(filename)
    
    top_k_file = f'{predictions_top_k}/{filename}'
    
    !tl get-kg-links -c $final_score_column -k 5 --k-rows $file > $top_k_file

84575189_0_6365692015941409487.csv
get-kg-links-siamese_prediction Time: 0.7082772254943848s
28086084_0_3127660530989916727.csv
get-kg-links-siamese_prediction Time: 1.5682003498077393s
50270082_0_444360818941411589.csv
get-kg-links-siamese_prediction Time: 1.1901907920837402s
29414811_2_4773219892816395776.csv
get-kg-links-siamese_prediction Time: 0.16732096672058105s
39759273_0_1427898308030295194.csv
get-kg-links-siamese_prediction Time: 0.7116034030914307s
14380604_4_3329235705746762392.csv
get-kg-links-siamese_prediction Time: 0.1615145206451416s
14067031_0_559833072073397908.csv
get-kg-links-siamese_prediction Time: 0.4001154899597168s
45073662_0_3179937335063201739.csv
get-kg-links-siamese_prediction Time: 0.20262742042541504s
52299421_0_4473286348258170200.csv
get-kg-links-siamese_prediction Time: 0.6912147998809814s


### Colorize the predicted files

In [12]:
for file in glob.glob(predictions_top_k+"/*.csv"):
    filename = file.split("/")[-1]
    
    print(filename)
    
    colorized = f'{colorized_path}/{filename.strip(".csv")}.xlsx'
    
    !tl add-color -c "$final_score_column,evaluation_label" -k 5 $file --output "$colorized"

84575189_0_6365692015941409487.csv
add-color Time: 0.15748333930969238s
28086084_0_3127660530989916727.csv
add-color Time: 0.3238708972930908s
50270082_0_444360818941411589.csv
add-color Time: 0.23525667190551758s
29414811_2_4773219892816395776.csv
add-color Time: 0.06061553955078125s
39759273_0_1427898308030295194.csv
add-color Time: 0.1501917839050293s
14380604_4_3329235705746762392.csv
add-color Time: 0.05891609191894531s
14067031_0_559833072073397908.csv
add-color Time: 0.09913015365600586s
45073662_0_3179937335063201739.csv
add-color Time: 0.06687641143798828s
52299421_0_4473286348258170200.csv
add-color Time: 0.13943266868591309s


### Measure the metrics for the files

In [13]:
df_list = []
for file in glob.glob(predictions_top_k+"/*.csv"):
    filename = file.split("/")[-1]
    
    print(filename)
    
    metrics_file = f'{metrics_path}/{filename}'
    
    !tl metrics $file -k 5 -c $final_score_column --tag $filename> $metrics_file
    
    df_list.append(pd.read_csv(metrics_file))
    
metrics_df = pd.concat(df_list)

84575189_0_6365692015941409487.csv
metrics Time: 0.5001442432403564s
28086084_0_3127660530989916727.csv
metrics Time: 0.9671831130981445s
50270082_0_444360818941411589.csv
metrics Time: 0.7799386978149414s
29414811_2_4773219892816395776.csv
metrics Time: 0.14821624755859375s
39759273_0_1427898308030295194.csv
metrics Time: 0.4858057498931885s
14380604_4_3329235705746762392.csv
metrics Time: 0.1184241771697998s
14067031_0_559833072073397908.csv
metrics Time: 0.29844236373901367s
45073662_0_3179937335063201739.csv
metrics Time: 0.15725111961364746s
52299421_0_4473286348258170200.csv
metrics Time: 0.4804060459136963s


In [14]:
metrics_df

,k,f1,precision,recall,tag
0,5,0.942526,0.908163,0.979592,84575189_0_6365692015941409487.csv
0,5,0.841411,0.804545,0.881818,28086084_0_3127660530989916727.csv
0,5,0.959752,0.922619,1.000000,50270082_0_444360818941411589.csv
0,5,0.926829,0.863636,1.000000,29414811_2_4773219892816395776.csv
0,5,0.990000,0.990000,0.990000,39759273_0_1427898308030295194.csv
0,5,1.000000,1.000000,1.000000,14380604_4_3329235705746762392.csv
0,5,0.981132,0.981132,0.981132,14067031_0_559833072073397908.csv
0,5,0.981132,0.962963,1.000000,45073662_0_3179937335063201739.csv
0,5,0.978022,0.978022,0.978022,52299421_0_4473286348258170200.csv


In [16]:
metrics_df.to_csv(metrics_path+"/metrics_top_5.csv", index=False)